### utils.py

Clears the terminal screen.

In [ ]:
import os

def clear_screen():
    """Clears the terminal screen."""
    os.system('cls' if os.name == 'nt' else 'clear')

### globals.py



In [ ]:
ROWS = None
COLS = None
MINES = None

### print_board.py
```
board =[
             [('0','?? '),('0','?? '),('0',"?  "),('0',"?  "),('0',' '),('0','5')],    
             [('1','0'),('1','1'),('1','2'),('1','3'),('1','4'),('1','5')],   
             [('2','0'),('2','1'),('2','2'),('2','3'),('2','4'),('2','5')],     
    ] 

The above representation is actually a 2D board with a 2 element  
tuple in each column. Each row has 6 tuples 

We could also represent the board as a true 3D: 

board = [ 
           [ [ ('0', '??') ], [ (' ', '  ') ], [ (' ', '  ') ], [ ('0', '??') ], [ (' ', '  ') ], [ (' ', '  ') ] ], 
           [ [ ('0', '??') ], [ (' ', '  ') ], [ (' ', '  ') ], [ ('0', '??') ], [ (' ', '  ') ], [ (' ', '  ') ] ], 
           [ [ ('0', '??') ], [ (' ', '  ') ], [ (' ', '  ') ], [ ('0', '??') ], [ (' ', '  ') ], [ (' ', '  ') ] ] 
        ] 
```
The depth dimension is a tuple in each column.  
Each row has 6 column lists and each column has a tuple. 

globals.COLS = 6
globals.ROWS = 3

In [ ]:
import globals
def print_board(board: list, level: int):
    
    line_hash = '|-----'    

    print('      ', end = '')        
    for idx in range(globals.COLS):
        print(f'   {idx}  ', end = '')

    print(f'\n      {line_hash * globals.COLS}|')    

    for row in range(globals.ROWS):
        print(f'  {row}   ', end = '') 
        for col in range(globals.COLS): 
            symbol = board[row][col][level]

            if symbol == '??':
                print(f'| {symbol:3}' , end = '')
            else: 
                print(f'| {symbol:3} ' , end = '')            
        print('|')

        print(f'      {line_hash * globals.COLS}|')

### get_validated_input.py
```
KEY INSIGHT: 
In the game we need to get 5 inputs. 
    1) height of board
    2) width of board
    3) number of mines
    4) column selection
    5) row selection

We are always asking for an integer and need to check
input falls within range. Rather than write the same 
code 5 times we can create a generic function.

Prompt the user for an integer input within a specified range.

Continuously prompts the user using `txt1` until a valid integer within
the range [`start`, `finish`] is entered. If the input is invalid,
`txt2` is shown in subsequent prompts until valid input is received.
```

In [ ]:
def get_validated_input(txt1, txt2, low_int, high_int):

    txt = txt1
    while True:
        try:
            value = int(input(txt))
            if not (low_int <= value <= high_int): raise ValueError
            break
        except ValueError:
            txt = txt2

    return value

### initialize_board.py
```
Before we start playing the game we need to initalize the board. Let's
consider the player board to be the first element in the tuple and the 
base board to be the second element in the tuple. To initalize the player 
board we need to fill it with diamonds. To iniitalize the base board we
need to fill it first with random mines, and then for each other cell fill
it with number of adjacent mines or empty space for 0. We use '   ' for 
0 or a blank so formatting in print_board works correctly.
```

In [ ]:
import globals
import utils
from get_validated_input import get_validated_input
from place_random_mines import place_random_mines
from count_adjacent_bombs import count_adjacent_bombs

def initialize_board():
    
    utils.clear_screen() 

    txt1 = "Enter the width of the board: "
    txt2 = "width must be between 2 and 10 (inclusive), re-enter: "  
    low = 2
    high = 10  
    globals.COLS = get_validated_input(txt1, txt2, low, high)

    txt1 = "Enter the height of the board: "
    txt2 = "Height must be between 2 and 10 (inclusive), re-enter: "    
    globals.ROWS = get_validated_input(txt1, txt2, low, high)

    txt1 = "Enter the number of random mines: "
    low = 1
    high = globals.ROWS * globals.COLS - 1
    txt2 = f"Mines must be between 1 and {high} (inclusive), re-enter: "    
    globals.MINES = get_validated_input(txt1, txt2, low, high)   

    board = []     
    for _ in range(globals.ROWS):
        row = []                      
        for _ in range(globals.COLS):
            col = (" \u2666" , '   ')  
            row.append(col)            
        board.append(row)              

    board = place_random_mines(board)
    board = count_adjacent_bombs(board)
    return board

### place_random_mines.py
```
To place random mines we use random.randint to get col and row.
We then check if board[row][col][1] is empty. We need to reassign
the tuple so both the first and second element need to be assigned.
We do this in a while loop until we've placed globals.mines on the board.
```

In [ ]:
import globals
import random
def place_random_mines(board:list):

    cnt = 0
    while cnt < globals.MINES:
        col = random.randint(0, globals.COLS - 1)
        row = random.randint(0, globals.ROWS - 1)
        if board[row][col][1] == '   ':
            board[row][col] =  (board[row][col][0], '\U0001F4A3')
            cnt += 1
    return board

### is_bomb_at.py
```
this routine checks if there is a mine/bomb at given
coordinates on the base board
```

In [ ]:
def is_bomb_at(board: list, coord: tuple) -> bool:

    bomb = False
    if board[coord[0]][coord[1]][1] == '\U0001F4A3': bomb = True    
    return bomb

### get_adjacent_cells.py
```
Returns a list of valid board coordinates adjacent to a given cell.

The result includes the given coordinate itself and all surrounding 
cells (up to 8 neighbors). Coordinates that would fall outside the 
board are excluded.
```

In [ ]:
import globals
def get_adjacent_cells(coord: tuple) -> list:

    squares = []
    adjustments = [(-1, -1), (-1, 0), (-1, 1),
                   ( 0, -1), ( 0, 0), ( 0, 1),
                   ( 1, -1), ( 1, 0), ( 1, 1)]

    for dr, dc in adjustments:
        new_row = coord[0] + dr
        new_col = coord[1] + dc
        if 0 <= new_row < globals.ROWS and 0 <= new_col < globals.COLS:
            squares.append((new_row, new_col))
    return squares

### count_adjacent_bombs.py
```
Now that bombs/mines have been placed on the board we check every
cell on the board. If the cell is a bomb/mine we skip over it.
We call get_adjacent_cells and get back a list of cell coordinates.
We remove the current cell from the list and count the number of bombs/
mines we have in adjacent cells.
```

In [ ]:
from is_bomb_at import is_bomb_at
from get_adjacent_cells import get_adjacent_cells
import globals
def count_adjacent_bombs(board: list ) -> list:
    
    for r in range(globals.ROWS):
        for c in range(globals.COLS):
            coord = (r, c)
            if is_bomb_at(board, (r, c)): continue
            squares = get_adjacent_cells(coord)
            squares = [s for s in squares if s != (r, c)]
            cnt = 0
            for square in squares:
                if board[square[0]][square[1]][1] == '\U0001F4A3' : cnt += 1    
            if cnt != 0:
                board[r][c] = (board[r][c][0], cnt)
            else:
                board[r][c] = (board[r][c][0], '   ')
            board[r][c] = (board[square[0]][square[1]][0], cnt) if cnt != 0 else (board[square[0]][square[1]][0], '   ')
    return board

### game_over.py
```
The game is over when either a cell is choosen that has bomb/mine
on it or when the number of diamnonds on the player board is
equal globals.MINES. We check the first condition directly
when we play the game and the latter condition here.
```

In [ ]:
import globals
def game_over(board:list) -> bool:
    
    cnt = 0
    for row in range(globals.ROWS):
        for col in range(globals.COLS):
            if board[row][col][0]  == " \u2666" : cnt += 1
    if cnt == globals.MINES: return True
    else: return False

### update_board.py
```
update_board involves the most computational thinking. When a cell is 
uncovered on the player's board we check what that cell is on the base
board.  It can be one of three things. 1) a bomb/mine, 2) a number or
3) blank space. If it is a bomb/mine play_minesweep will terminate the
game before we get here. If the base board cell is (2) or (3) we set the 
player board's corrosponding cell to base board's cell.

Here's where it get's interesting. If the cell is a blank we know all 
surrounding cells do not contain a bomb/mine. All of those can be
uncovered.  If any of those cells are blank we need to uncover their
adjacent cells.  This process repeats untill all 'connected' black spaces
are uncovered.
```

In [ ]:
from get_adjacent_cells import get_adjacent_cells
def update_board(board: list, coord: tuple):

    stack = []
    stack.append(coord)

    while stack:
        row, col = stack.pop(0)
        board[row][col] = (board[row][col][1], board[row][col][1])
        if board[row][col][0] == "   ":        
            squares = get_adjacent_cells((row, col))
            squares = [s for s in squares if s != (row, col)]
            for square in squares:
                r, c = square
                if board[r][c][0] != board[r][c][1]:
                    stack.append((r, c))

### play_minesweep.py
```
Before we can begin to play we need to initalize the board. Then
enter a while loop until the game is over. Get input from user and 
end the game if bomb/mine choosen. If not, update board'
```

In [ ]:
import globals
import utils
from initialize_board import initialize_board
from print_board import print_board
from game_over import game_over
from get_validated_input import get_validated_input
from is_bomb_at import is_bomb_at
from update_board import update_board

def play_minesweep():


    board = initialize_board()

    utils.clear_screen()   
    print()   

    while not game_over(board):
        print_board(board, 0)
        print()

        txt1 = "How many over do you want to dig: "
        txt2 = f"You have to stay on the board. Enter 0 - {globals.COLS - 1}: "  
        col = get_validated_input(txt1, txt2, 0, globals.COLS - 1)

        txt1 = "How many down do you want to dig: "
        txt2 = f"You have to stay on the board. Enter 0 - {globals.ROWS - 1}: "  
        row = get_validated_input(txt1, txt2, 0, globals.ROWS - 1)

        if is_bomb_at(board, (row, col)):
            print('\n\nYou lost! You blew up the board!\n')            
            print_board(board, 1)
            print() 
            break

        update_board(board, (row, col))

    if game_over(board):
        print('\nYou won, congratulatons!\n\n') 
        print_board(board, 0)

play_minesweep()